# Preprocesamiento para Power BI

Este notebook es muy similar al de eda.ipynb, las diferencias se encuentan en que la columna year_week no es eliminada, y no se han creado columnas de year, month, day de manera indivual a partir de la columna date.

## 1. Libraries


In [ ]:
# Datos
import numpy as np
import pandas as pd

# Visualizacion
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from random import random

# Preprocessing
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.feature_selection import VarianceThreshold
from scipy import stats

# Evaluacion
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import metrics

#warnings
import warnings
warnings.filterwarnings("ignore")

#Extras
import os
import pickle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 2. Dataset

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/TFM/CSV/creation/gb_union_weeks.csv')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.shape

### Optimización de los datos

In [ ]:
# Columnas a convertir a 'category', ya que son variables con pocos valores únicos (optimizar memoria)
col_cat= ['category', 'store', 'store_code', 'region', 'department']

# Aplicar el bucle para convertir las columnas a 'category'
for i in col_cat:
    df[i] = df[i].astype('category')

In [ ]:
df['date'] = pd.to_datetime(df["date"])

In [ ]:
df['quantity']=df['quantity'].astype('int16')

In [ ]:
df.info()

## 3. Preprocessing

In [ ]:
df[df.duplicated(keep=False)]

In [ ]:
df.isnull().sum()

In [ ]:
# Ordenar el DataFrame por 'id' y 'date'
df = df.sort_values(by=['id', 'date'])

In [ ]:
df.head()

Este código recorre el DataFrame fila por fila. Si encuentra un 'quantity' distinto de cero para un 'id', pasa al siguiente 'id'. Si encuentra un 'quantity' igual a cero y aún no ha encontrado un valor distinto de cero para ese 'id', entonces reemplaza el valor de 'sell_price' por -99.

Este código se ha realizado porque se observaron que algunos productos al inicio de los datos no tenían ventas. Por lo tanto, se concluyó que salieron a mercado un tiempo más tarde, pero que sufieron épocas que no fueron adquiridos por los clientes finales.


In [ ]:
previous_id = None
for index, row in df.iterrows():
    current_id = row['id']
    if current_id != previous_id:
        previous_id = current_id
        found_nonzero = False
    if row['quantity'] != 0:
        found_nonzero = True
    if row['quantity'] == 0 and not found_nonzero:
        df.at[index, 'sell_price'] = -99

print(df)

In [ ]:
df.isnull().sum()

In [ ]:
df["sell_price"] = df.groupby("id", group_keys = False)["sell_price"].apply(
    lambda series: series.backfill().ffill()
)

In [ ]:
df['sell_price'] = df['sell_price'].round(2)

In [ ]:
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
df = df[df['sell_price'] != -99]

print(df)

In [ ]:
# Reasignar los índices después de eliminar las filas
df.reset_index(drop=True, inplace=True)

## 4. Feature engineering

In [ ]:
df['revenue'] = df['quantity'] * df['sell_price']
df.head()

In [ ]:
df.info()

In [ ]:
df['revenue']=df['revenue'].astype('float32')

In [ ]:
df.info()

## 5. CSV

In [ ]:
df.to_csv('/content/drive/MyDrive/Colab Notebooks/TFM/CSV/creation/preprocessing_powerbi.csv', index=False)